In [ ]:
! pip install pandas
! pip install PyPortfolioOpt

In [4]:
from dotenv import load_dotenv

load_dotenv();

In [5]:
import requests

iex_env = os.environ.get("IEX_ENV", "cloud")
iex_token = os.environ.get("IEX_TOKEN", "")

def get_prices(symbol, date_param='5d'):

    url_prefix = f"https://{iex_env}.iexapis.com/stable/"

    path = f'stock/{symbol}/chart/{date_param}?chartCloseOnly=True&&token={iex_token}'
    print(f"Fetching {date_param} data for {symbol}")
    full_url = requests.compat.urljoin(url_prefix, path)

    try:
        resp = requests.get(full_url)
    except Exception as e:
        print(f"Exception {e} occurred!")
        return None

    if resp.status_code != 200:
        print(f"Uh oh, something's wrong! Response code {resp.status_code} received.")
        return resp

    else:
        print(f"Got the data")
        return resp

In [6]:
import os
import json

def get_cached_prices(symbol, date_range, outpath):
    if not os.path.exists(outpath):
        resp = get_prices(symbol, date_param=date_range)
        if resp is not None:
            prices_obj = json.loads(resp.text)
            with open(outpath, "w") as f:
                json.dump(prices_obj, f)
            return prices_obj
    else:
        with open(outpath, "r") as f:
            data_obj = json.load(f)
        return data_obj

In [7]:
symbols = ["MSFT", "AAPL", "NVDA", "JNJ", "KHC", "ALL"]

In [8]:
def getDf(symbol, data):
    df = pd.DataFrame(data)
    df = df[['date', 'close']]
    df['date'] = pd.to_datetime(df.date)
    df.columns = ['date', symbol]
    df.set_index('date')
    return df;

In [ ]:
import pandas as pd

date_range = "5y"

symbols_dict = dict()

for symbol in symbols:
    outpath = f"./.data/stocks/{symbol}_{date_range}.json"
    symbols_dict[symbol] = get_cached_prices(symbol, date_range, outpath)

In [10]:
import utils

df = utils.symbol_dict_to_df(symbols_dict)
df['date'] = pd.to_datetime(df.date)
df = df.pivot_table(values='close', index='date', columns='symbol', aggfunc='last')
df

symbol,AAPL,ALL,JNJ,KHC,MSFT,NVDA
date,,,,,,
2017-03-10,35.2840,85.96,130.96,95.93,67.13,25.1000
2017-03-13,35.3000,84.70,130.78,91.76,66.48,25.8382
2017-03-14,35.0005,85.90,133.28,93.19,65.37,26.0420
2017-03-15,35.1480,86.67,134.64,93.14,67.80,26.1975
2017-03-16,35.4757,84.19,130.12,95.46,67.74,26.4165
...,...,...,...,...,...,...
2022-03-02,173.3500,125.24,168.84,39.26,310.56,251.2000
2022-03-03,168.2100,130.59,174.35,41.81,303.21,246.5800
2022-03-04,168.5300,133.29,170.97,41.17,297.25,238.7200


In [11]:
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting

mu = expected_returns.mean_historical_return(df, frequency=252)

In [12]:
S = risk_models.sample_cov(df, frequency=252)

In [13]:
ef = EfficientFrontier(mu, S)

In [14]:
weights = ef.min_volatility()
weights

OrderedDict([('AAPL', 0.1378645379815755),
             ('ALL', 0.2279367134048076),
             ('JNJ', 0.2812347586318125),
             ('KHC', 0.1732130572451437),
             ('MSFT', 0.1566960269138146),
             ('NVDA', 0.023054905822846)])

In [15]:
ef.portfolio_performance(verbose=True)

Expected annual return: 12.2%
Annual volatility: 24.4%
Sharpe Ratio: 0.42


(0.12165003373853475, 0.24443386184033192, 0.4158590506782328)